In [14]:
import pandas as pd

import io
from google.colab import files

In [ ]:
uploaded = files.upload()

In [74]:
# Load and clean the data
data = pd.read_csv(io.BytesIO(uploaded['dataset_CO2_emission_of_cars.csv']))

if set(data.all()) == {True} and set(data.all(axis ='columns')) == {True}:
  print("Data is Complete")

make_dict = {}
i = 1
for unique_make in set(data['Make']):
  make_dict[unique_make] = i
  i = i + 1
  
model_dict = {}
i = 1
for unique_model in set(data['Model']):
  model_dict[unique_model] = i
  i = i + 1

vehicle_class_dict = {}
i = 1
for unique_vehicle_class in set(data['Vehicle Class']):
  vehicle_class_dict[unique_vehicle_class] = i
  i = i + 1

transmission_dict = {}
i = 1
for unique_transmission in set(data['Transmission']):
  transmission_dict[unique_transmission] = i
  i = i + 1

fuel_type_dict = {}
i = 1
for unique_fuel_type in set(data['Fuel Type']):
  fuel_type_dict[unique_fuel_type] = i
  i = i + 1

data_clean = data
data_clean['Make'] = data_clean['Make'].map(make_dict, 0)
data_clean['Model'] = data_clean['Model'].map(model_dict, 0)
data_clean['Vehicle Class'] = data_clean['Vehicle Class'].map(vehicle_class_dict, 0)
data_clean['Transmission'] = data_clean['Transmission'].map(transmission_dict, 0)
data_clean['Fuel Type'] = data_clean['Fuel Type'].map(fuel_type_dict, 0)

data_clean.head()

Data is Complete


,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,10,483,1,2.0,4,1,2,9.9,6.7,8.5,33,196
1,10,483,1,2.4,4,22,2,11.2,7.7,9.6,29,221
2,10,1538,1,1.5,4,3,2,6.0,5.8,5.9,48,136
3,10,1869,15,3.5,6,10,2,12.7,9.1,11.1,25,255
4,10,436,15,3.5,6,10,2,12.1,8.7,10.6,27,244


In [76]:
# Feature Selection

# Categorical Variables
data_clean['Make'] = data_clean['Make'].astype('category')
dum = pd.get_dummies(data_clean.Make)
data_clean = data_clean.drop(columns=['Make']).merge(dum, left_index=True, right_index=True)

X = data_clean.drop(columns='CO2 Emissions(g/km)')
Y = data_clean['CO2 Emissions(g/km)']